In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
 
from matplotlib import pyplot as plt
 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('final_data.csv', delimiter=';')
data.head(5)

,Unnamed: 0,address,z_address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,usecode,yearbuilt,zestimate,zindexvalue,zipcode,zpid
0,2,Address: 1160 Mission Street #2007,1160 Mission St UNIT 2007,2.0,2.0,1043.0,02/17/2016,1300000.0,37.778705,-122.412635,South of Market,4.0,Condominium,2007.0,1167508.0,"975,700",94103.0,83152781.0
1,5,Address: 260 King Street #475,260 King St UNIT 475,1.0,1.0,903.0,02/17/2016,750000.0,37.777641,-122.393417,South of Market,3.0,Condominium,2004.0,823719.0,"975,700",94107.0,69819817.0
2,7,Address: 560 Missouri Street #B,560 Missouri St # B,4.0,3.0,1425.0,02/17/2016,1495000.0,37.759198,-122.396516,Potrero Hill,6.0,Condominium,2003.0,1708594.0,"1,277,600",94107.0,64972847.0
3,9,Address: 350 Missouri Street,350 Missouri St,3.0,3.0,2231.0,02/17/2016,2700000.0,37.761886,-122.396769,Potrero Hill,10.0,SingleFamily,1927.0,2411236.0,"1,277,600",94107.0,15149489.0
4,11,Address: 3658 Folsom Street,3658 Folsom St,3.0,3.0,1300.0,02/17/2016,1530000.0,37.740795,-122.413453,Bernal Heights,4.0,SingleFamily,1900.0,1918539.0,"1,248,000",94110.0,15161978.0


In [5]:
data.shape

(11330, 18)

Будем использовать линейную регрессию для тестирования качества данных

In [6]:
def get_score(X,y, random_seed=42, model=None):
  if model is None:
    model = LinearRegression()
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed )  
  model.fit(X_train, y_train)
  return model.score(X_test, y_test)

Для поиска выбросов будем использовать boxplot

In [18]:
def get_boxplot(X, columns=[]):
    for i in columns:
        sns.boxplot(x=X[i])
    pass  

def get_pairplot(X, columns=None):
    if columns is None:
        columns = list(X.columns)
    sns.pairplot(X[columns])
    pass    

Для заполнения пропущенных значений будем использовать стандартные способы
- заполнение значением
- max
- min
- mode
- median
- mean
- метод индикатора
- линейная регрессия

In [19]:
def get_value(X, column, mode='mean', value=0, columns_for_reg=None):
  if mode == 'value':
    X.loc[X[X[column].isna()].index ,column] = value
  elif mode == 'max':
    X.loc[X[X[column].isna()].index ,column] = X[column].max()
  elif mode == 'min':
    X.loc[X[X[column].isna()].index ,column] = X[column].min()
  elif mode == 'median':
    X.loc[X[X[column].isna()].index ,column] = X[column].median()    
  elif mode == 'mode':
    X.loc[X[X[column].isna()].index, column] = X[column].mode()[0]  
  elif mode == 'indicator':
    X.loc[X[X[column].isna()].index, column] = 0
    X['ind_'+str(column)] = 0
    X.loc[X[X[column].isna()].index, 'ind_'+str(column)] = 1
  elif mode == 'linreg':
    if columns_for_reg is None:
      cols = list(X.select_dtypes([np.number]).columns) 
      cols.remove(column)
    else:
      cols = columns_for_reg
    X_tmp = X.dropna()  
    m = LinearRegression().fit(X_tmp[cols], X_tmp[column])
    X.loc[X[X[column].isna()].index, column] = m.predict(X[X[column].isna()][cols])
  else:
    X.loc[X[X[column].isna()].index, column] = X[column].mean()
  return X  


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11330 entries, 0 to 11329
Data columns (total 18 columns):
Unnamed: 0       11330 non-null int64
address          11330 non-null object
z_address        11330 non-null object
bathrooms        11330 non-null float64
bedrooms         11330 non-null float64
finishedsqft     11211 non-null float64
lastsolddate     11330 non-null object
lastsoldprice    11330 non-null float64
latitude         11315 non-null float64
longitude        11315 non-null float64
neighborhood     11330 non-null object
totalrooms       11281 non-null float64
usecode          11290 non-null object
yearbuilt        11330 non-null float64
zestimate        11330 non-null float64
zindexvalue      11330 non-null object
zipcode          11330 non-null float64
zpid             11330 non-null float64
dtypes: float64(11), int64(1), object(6)
memory usage: 1.6+ MB


In [10]:
data.describe()

,Unnamed: 0,bathrooms,bedrooms,finishedsqft,lastsoldprice,latitude,longitude,totalrooms,yearbuilt,zestimate,zipcode,zpid
count,11330.000000,11330.000000,11330.000000,11211.000000,1.133000e+04,11315.000000,11315.000000,11281.000000,11330.000000,1.133000e+04,11330.000000,1.133000e+04
mean,9171.729214,1.980229,2.614475,1586.081349,1.263928e+06,37.759715,-122.436508,6.113997,1948.498147,1.565695e+06,94116.912004,3.689973e+07
std,4921.941074,1.047358,1.299457,922.815803,1.042079e+06,0.025583,0.030742,12.151195,37.911196,1.229417e+06,9.400877,7.800741e+07
min,2.000000,0.500000,0.000000,1.000000,5.350000e+02,37.708170,-122.510726,1.000000,1860.000000,4.323850e+05,94102.000000,1.506329e+07
25%,5039.750000,1.000000,2.000000,1019.000000,7.292500e+05,37.739284,-122.455149,4.000000,1916.000000,9.052375e+05,94110.000000,1.510847e+07
50%,9198.500000,2.000000,2.000000,1362.000000,9.900000e+05,37.760514,-122.432510,5.000000,1940.000000,1.230758e+06,94115.000000,1.515697e+07
75%,13374.750000,2.000000,3.000000,1878.000000,1.450000e+06,37.781386,-122.413353,7.000000,1986.000000,1.731170e+06,94123.000000,5.970040e+07
max,17632.000000,14.000000,20.000000,27275.000000,2.388900e+07,37.806083,-122.381201,1264.000000,2016.000000,1.553325e+07,94158.000000,2.146999e+09


In [11]:
data.columns

Index(['Unnamed: 0', 'address', 'z_address', 'bathrooms', 'bedrooms',
       'finishedsqft', 'lastsolddate', 'lastsoldprice', 'latitude',
       'longitude', 'neighborhood', 'totalrooms', 'usecode', 'yearbuilt',
       'zestimate', 'zindexvalue', 'zipcode', 'zpid'],
      dtype='object')

In [12]:
cols = ['Unnamed: 0', 'bathrooms', 'bedrooms',
       'finishedsqft', 'lastsoldprice', 'latitude',
       'longitude', 'totalrooms','yearbuilt','zipcode']
cols2 = ['Unnamed: 0', 'bathrooms', 'bedrooms',
       'finishedsqft', 'latitude',
       'longitude', 'totalrooms','yearbuilt','zipcode']

In [13]:
data.dropna().shape

(11108, 18)

In [14]:
# -             = 0.8375491685999106, 0.607156739569225
# finishedsqft  = 0.8524615199933394, 0.5577844171230759
# long and lat  = 0.8517111453083177, 0.6874289038247563
# totalrooms    = 0.8598254992362179, 0.6426676296618906

SyntaxError: invalid syntax (<ipython-input-14-f98496e9d777>, line 1)

In [22]:
get_score(data.dropna()[cols], data.dropna()['zestimate'])

0.8375491685999527

In [23]:
get_score(data.dropna()[cols2], data.dropna()['zestimate'])

0.6071567395692328

In [24]:
for_drop = ['Unnamed: 0', 'address', 'zindexvalue', 'zpid']
target = ['zestimate']

In [25]:

_ = get_value(data, 'finishedsqft', mode='linreg', value=0, columns_for_reg=['bathrooms', 'bedrooms', 'lastsoldprice','totalrooms'])

In [36]:
data['neighborhood'].value_counts().head()

Mission            540
Bernal Heights     525
South of Market    524
South Beach        461
Pacific Heights    439
Name: neighborhood, dtype: int64

In [27]:
for i, el in data[data['latitude'].isna()].iterrows():
  data.loc[i, 'longitude'] = data[data['neighborhood'] == el['neighborhood']]['longitude'].mean()
  data.loc[i, 'latitude'] = data[data['neighborhood'] == el['neighborhood']]['latitude'].mean()


In [28]:
data[data.totalrooms - (data.bedrooms + data.bathrooms) < 0]

,Unnamed: 0,address,z_address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,usecode,yearbuilt,zestimate,zindexvalue,zipcode,zpid
2,7,Address: 560 Missouri Street #B,560 Missouri St # B,4.00,3.0,1425.000000,02/17/2016,1495000.0,37.759198,-122.396516,Potrero Hill,6.0,Condominium,2003.0,1708594.0,"1,277,600",94107.0,6.497285e+07
4,11,Address: 3658 Folsom Street,3658 Folsom St,3.00,3.0,1300.000000,02/17/2016,1530000.0,37.740795,-122.413453,Bernal Heights,4.0,SingleFamily,1900.0,1918539.0,"1,248,000",94110.0,1.516198e+07
39,74,Address: 2455 14th Avenue,2455 14th Ave,3.75,4.0,1846.000000,02/10/2016,1525000.0,37.742271,-122.470518,West Portal,5.0,SingleFamily,1932.0,1195679.0,"1,565,000",94116.0,1.512286e+07
46,93,Address: 784 Spruce Street,784-786 Spruce St,3.00,8.0,2600.000000,02/09/2016,1535000.0,37.780606,-122.452663,Lone Mountain,10.0,MultiFamily2To4,1927.0,1524759.0,"1,447,700",94118.0,1.508382e+07
60,121,Address: 2017 Buchanan Street,2017 Buchanan St,4.00,4.0,2850.000000,02/05/2016,2800000.0,37.788913,-122.430728,Lower Pacific Heights,7.0,Condominium,1890.0,3032479.0,"1,178,700",94115.0,2.169907e+07
94,253,Address: 701 Pennsylvania Avenue,701 Pennsylvania Ave # T13,2.00,1.0,842.000000,03/24/2015,640000.0,37.757380,-122.392830,Potrero Hill,2.0,Condominium,1999.0,905401.0,"1,277,600",94107.0,5.877020e+07
97,256,Address: 164 Seville Street,164 Seville St,3.00,3.0,2234.000000,01/26/2016,1125000.0,37.711727,-122.437085,Crocker Amazon,3.0,SingleFamily,1924.0,917185.0,"823,000",94112.0,1.517815e+07
98,257,Address: 145 Alpine Terrace,145-147 Alpine Ter,5.50,6.0,4749.000000,01/26/2016,5100000.0,37.768186,-122.437144,Buena Vista Park,10.0,SingleFamily,2007.0,4791629.0,"1,500,900",94117.0,8.924057e+07
111,296,Address: 712 Dorado Terrace #53,712 Dorado Ter # 53,2.50,2.0,1398.579546,01/22/2016,780000.0,37.725575,-122.461600,Mount Davidson Manor,4.0,Condominium,1985.0,773173.0,"1,378,600",94112.0,1.514233e+07
113,298,Address: 4450 24th Street,4450 24th St,3.00,4.0,2500.000000,01/22/2016,1400000.0,37.751072,-122.441843,Noe Valley,5.0,SingleFamily,1900.0,2088377.0,"1,782,000",94114.0,1.513224e+07


In [29]:
get_score(data.dropna()[['finishedsqft']], data.dropna()['zestimate'])

0.6064430886914685

In [30]:

_ = get_value(data, 'totalrooms', mode='linreg', value=0, columns_for_reg=['bathrooms', 'bedrooms', 'lastsoldprice','finishedsqft'])

In [31]:
data['totalrooms'] = data['totalrooms'].apply(lambda x: round(x))

In [32]:
data['usecode'].value_counts(dropna=False) 

SingleFamily        5785
Condominium         4782
MultiFamily2To4      484
Duplex               146
Townhouse             66
NaN                   40
Miscellaneous         17
Cooperative            3
Apartment              3
MultiFamily5Plus       2
Mobile                 2
Name: usecode, dtype: int64

In [33]:
_ = get_value(data, 'usecode', mode='value', value='other')

In [34]:
data.head(2)

,Unnamed: 0,address,z_address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,usecode,yearbuilt,zestimate,zindexvalue,zipcode,zpid
0,2,Address: 1160 Mission Street #2007,1160 Mission St UNIT 2007,2.0,2.0,1043.0,02/17/2016,1300000.0,37.778705,-122.412635,South of Market,4,Condominium,2007.0,1167508.0,"975,700",94103.0,83152781.0
1,5,Address: 260 King Street #475,260 King St UNIT 475,1.0,1.0,903.0,02/17/2016,750000.0,37.777641,-122.393417,South of Market,3,Condominium,2004.0,823719.0,"975,700",94107.0,69819817.0


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11330 entries, 0 to 11329
Data columns (total 18 columns):
Unnamed: 0       11330 non-null int64
address          11330 non-null object
z_address        11330 non-null object
bathrooms        11330 non-null float64
bedrooms         11330 non-null float64
finishedsqft     11330 non-null float64
lastsolddate     11330 non-null object
lastsoldprice    11330 non-null float64
latitude         11330 non-null float64
longitude        11330 non-null float64
neighborhood     11330 non-null object
totalrooms       11330 non-null int64
usecode          11330 non-null object
yearbuilt        11330 non-null float64
zestimate        11330 non-null float64
zindexvalue      11330 non-null object
zipcode          11330 non-null float64
zpid             11330 non-null float64
dtypes: float64(10), int64(2), object(6)
memory usage: 1.6+ MB


In [ ]:
#data.to_csv('new_data.csv')

 ## Вторая часть

In [ ]:
data = pd.read_csv('new_data.csv')

In [ ]:
def get_one_hot(X, cols):
    for each in cols:
        dummies = pd.get_dummies(X[each], prefix=each, drop_first=False)
        X = pd.concat([X, dummies], axis=1)
    return X

def get_woe_v1(X, col, target_col):
    all_one = X[col].value_counts() / X[col].value_counts()
    all_good = len(X[X[target_col] == 1][col])
    all_bad = len(X[X[target_col] == 0][col])
    odds_series = (
        (((all_one*X[df_train[target_col] == 1][col].value_counts()).fillna(0)+0.5)/all_good)
        /
        (((all_one*X[X[target_col] == 0][col].value_counts()).fillna(0)+0.5)/all_bad)
    )
    category_woe_dict = np.log(odds_series).to_dict()
    X[col + '_woe'] = X[col].apply(category_woe_dict.get)
    return X

def get_woe_cols(X, cols, target_col):
    for col in cols:
        X = get_woe_v1(X, col, target_col)
    return X    

